In [1]:
import numpy as np
import pandas as pd
import scipy as sc

from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
data_init = pd.read_excel("C:/Users/Berezina.Darya/Documents/GitHub/Sberbank/total3.xlsx")

In [276]:
data = data_init.copy()
data["price_m"] = data.price_doc / data.full_sq
data["month"] = [int((d[:4])+(d[5:7])) for d in data.timestamp.values]

In [502]:
to_take = ["id", "full_sq", "life_sq", "floor", "max_floor", "build_year", "num_room",
          "state", "sub_area", 'kitch_sq','metro_min_walk',"month", "material", "kremlin_km", "price_doc", "price_m"]

In [503]:
df = data[to_take]
df = df.reset_index(drop=True)

In [504]:
df.head()

,id,full_sq,life_sq,floor,max_floor,build_year,num_room,state,sub_area,kitch_sq,metro_min_walk,month,material,kremlin_km,price_doc,price_m
0,1,43.0,27.0,4.0,9.0,1974.0,2.0,2.5,Bibirevo,6.0,13.575119,201108,1.0,15.156211,5850000,136046.511628
1,2,34.0,19.0,3.0,12.0,NaN,1.0,NaN,Nagatinskij Zaton,9.0,7.620630,201108,5.0,8.698054,6000000,176470.588235
2,3,43.0,29.0,2.0,NaN,NaN,NaN,NaN,Tekstil'shhiki,NaN,17.351515,201108,NaN,9.067885,5700000,132558.139535
3,4,89.0,50.0,9.0,NaN,NaN,NaN,NaN,Mitino,NaN,11.565624,201109,NaN,19.487005,13100000,147191.011236
4,5,77.0,77.0,4.0,NaN,NaN,NaN,NaN,Basmannoe,NaN,8.266305,201109,NaN,2.578671,16331452,212096.779221


In [505]:
df2 = df.copy()
df2.fillna(0, inplace=True)
df2 = df2[df2.price_doc > 0]
df2 = df2.drop(["id", "sub_area", "price_doc", "price_m"],1)

In [506]:
ss = StandardScaler()
df3 = ss.fit_transform(df2)

In [507]:
def find_similar(ind):
    v = df3[ind]
    id_best, dist = -1, 10000000
    for i in range(len(df3)):
        if i != ind:
            if sc.linalg.norm(df3[i]-v) < dist:
                dist = sc.linalg.norm(df3[i]-v)
                id_best = i
    return id_best

In [563]:
def find_similar(ind):
    v = df3[ind]
    r = []
    for i in range(len(df3)):
        if i != ind:
            r.append((i,sc.linalg.norm(df3[i]-v)))
    r.sort(key=lambda x: x[1], reverse=False)
    
    return r[:8]

In [784]:
df[(df.sub_area == "Babushkinskoe")&(df.month > 201400)]

,id,full_sq,life_sq,floor,max_floor,build_year,num_room,state,sub_area,kitch_sq,metro_min_walk,month,material,kremlin_km,price_doc,price_m
13852,13855,60.00,42.00,8.0,25.0,1993.0,2.0,4.0,Babushkinskoe,5.0,10.686848,201401,1.0,11.671381,14200000,236666.666667
13904,13907,79.00,48.00,12.0,14.0,1992.0,3.0,2.0,Babushkinskoe,8.0,10.686848,201401,1.0,11.671381,14600000,184810.126582
14571,14574,44.00,28.00,3.0,9.0,1970.0,2.0,NaN,Babushkinskoe,5.0,12.694497,201402,2.0,13.603345,7600000,172727.272727
14602,14605,29.00,17.00,4.0,9.0,1971.0,1.0,2.0,Babushkinskoe,5.0,10.118418,201402,2.0,12.320525,5600000,193103.448276
14612,14615,51.00,30.00,3.0,17.0,2008.0,2.0,2.0,Babushkinskoe,9.0,17.429841,201402,1.0,12.236399,3127850,61330.392157
14745,14748,42.00,29.00,5.0,5.0,1960.0,2.0,2.0,Babushkinskoe,5.0,19.837879,201402,2.0,12.300906,6450000,153571.428571
14795,14798,42.00,26.00,5.0,5.0,1961.0,2.0,2.0,Babushkinskoe,5.0,22.721681,201402,2.0,12.483441,5200000,123809.523810
15171,15174,56.00,41.00,1.0,9.0,1971.0,3.0,2.0,Babushkinskoe,5.0,10.118418,201402,2.0,12.320525,8380000,149642.857143
15287,15290,42.00,28.00,7.0,9.0,1970.0,2.0,2.0,Babushkinskoe,5.0,12.694497,201402,2.0,13.603345,7300000,173809.523810
15435,15438,39.00,23.00,10.0,12.0,1969.0,2.0,2.0,Babushkinskoe,6.0,17.176347,201402,1.0,12.197289,6940000,177948.717949


In [785]:
m = 13904

In [786]:
find_similar(m)

[(22272, 0.46672891662447874),
 (15273, 0.5030198477205295),
 (27018, 0.5373572561358033),
 (26825, 0.5668070345230872),
 (24204, 0.6067783788822471),
 (19372, 0.6156716176856656),
 (24183, 0.6170231397278546),
 (22624, 0.6179587828779046)]

In [796]:
df.loc[22624]

id                     22627
full_sq                   76
life_sq                   41
floor                     11
max_floor                 14
build_year              1987
num_room                   3
state                      2
sub_area          Novokosino
kitch_sq                   8
metro_min_walk        6.3157
month                 201409
material                   1
kremlin_km            15.528
price_doc           10300000
price_m               135526
Name: 22624, dtype: object

In [803]:
np.array([166197, 177632, 180723, 188356, 199333]).mean()

182448.20000000001

In [799]:
t = np.array([166197,199333,227027,188356,180723,177632,135526])
t.sort()
t

array([135526, 166197, 177632, 180723, 188356, 199333, 227027])

In [801]:
best, s = 0, np.std(t[:5])
for i in range(len(t)-5):
    if np.std(t[i:i+5]) < s:
        best = i

In [802]:
best

1

In [800]:
np.std(t)

26181.918323569054

In [654]:
comp = [(177419, 174173.39999999999), (213409, 165919.20000000001), (126154, 159173.0), (200000, 188236.20000000001),
    (212245, 198675.39999999999), (133621, 131460.20000000001), (145312, 151541.79999999999), (110925, 115923.8), 
        (263380,212883.60000000001), (305714,237811.0), (139344,159375.79999999999), (144444,148667.0), (224096,199459.0),
       (143352,111412.38), (300000,204248.0), (115637,112996.62), (160000,159146.60000000001), (192188,180141.20000000001),
       (250000,254665.75), (210959, 199190.20000000001), (184810, 182448.20000000001)]

In [534]:
df.loc[find_similar(m)]

id                          18957
full_sq                        32
life_sq                        20
floor                           1
max_floor                       5
build_year                   1963
num_room                        1
state                           2
sub_area          Sokolinaja Gora
kitch_sq                        5
metro_min_walk            9.71179
month                      201405
material                        1
kremlin_km                7.51437
price_doc                 5000000
price_m                    156250
Name: 18954, dtype: object

In [336]:
stop_list = []

for j in range(len(df3)):
    fs = find_similar(j)
    if df[["price_m"]].loc[j].values[0]/df[["price_m"]].loc[fs].values[0] < 0.5:
        stop_list.append(j)
    if df[["price_m"]].loc[j].values[0]/df[["price_m"]].loc[fs].values[0] > 2:
        stop_list.append(fs)

In [340]:
df = df[df.price_m > 0]

In [342]:
similar = []

for j in range(len(df)):
    similar.append(df[["id"]].loc[find_similar(j)].values[0])

In [344]:
df["similar"] = similar

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [382]:
res = df[df.build_year > 0]
res["stop_list"] = [1 if list(res.index)[i] in stop_list else 0 for i in range(len(res))]
res = res[res.stop_list == 0]
res.drop("stop_list",1,inplace=True)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [383]:
res.head()

,id,full_sq,life_sq,floor,max_floor,build_year,num_room,state,sub_area,kitch_sq,metro_min_walk,month,price_doc,price_m,similar
51,8814,39.0,29.0,4.0,12.0,1965.0,2.0,2.0,Kuncevo,0.0,29.227030,201307,7650000,196153.846154,11130
53,9127,38.0,18.0,3.0,17.0,1984.0,1.0,2.0,Kuncevo,8.0,17.792294,201307,7000000,184210.526316,10330
56,9893,58.0,32.0,16.0,17.0,1984.0,2.0,3.0,Kuncevo,10.0,17.792294,201308,9300000,160344.827586,12317
57,10330,37.0,18.0,6.0,16.0,1987.0,1.0,3.0,Kuncevo,9.0,22.892151,201309,6550000,177027.027027,12054
59,10568,76.0,45.0,16.0,16.0,1982.0,3.0,2.0,Kuncevo,10.0,17.792294,201309,13750000,180921.052632,12027


In [384]:
to_desc = ["full_sq","life_sq","floor","max_floor","build_year","num_room","state","sub_area","kitch_sq","metro_min_walk"]

In [385]:
res = res[res.price_m > 110000]

In [387]:
res.to_excel("res_sber.xlsx", index=False)

In [388]:
res = pd.read_excel("res_sber.xlsx")

In [389]:
res.head()

,full_sq,life_sq,floor,max_floor,build_year,state,kitch_sq,metro_min_walk,price_2
0,8,1,-2,-3,12,0,0,-5.006846,-0.294952
1,-1,0,3,-1,3,1,1,5.099857,-0.038996
2,15,2,-3,-1,10,0,1,5.481355,0.025188
3,-3,1,0,-4,-9,0,0,3.452489,0.046700
4,5,-2,-3,1,25,0,0,-13.455808,0.081571


In [367]:
from sklearn.linear_model import LinearRegression

In [391]:
X = res.drop(["price_2","life_sq", "kitch_sq", "floor"],1)
y = res[["price_2"]]

In [392]:
lr = LinearRegression()
lr.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [393]:
lr.score(X, y)

0.056088192143984306

In [380]:
lr.coef_

array([[  2.39886955e+04,   6.44043858e+02,  -6.77260751e+05,
         -1.17453446e+04,  -4.31721948e+03,   6.53132950e-01]])

In [381]:
X

,full_sq,max_floor,build_year,state,metro_min_walk,price_2
0,0.205128,-0.250000,0.006107,0.000000,-0.171309,138297.872340
1,-0.026316,-0.058824,0.001512,0.500000,0.286633,177027.027027
2,0.258621,-0.058824,0.005040,0.000000,0.308075,164383.561644
3,-0.081081,-0.250000,-0.004529,0.000000,0.150815,185294.117647
4,0.065789,0.062500,0.012614,0.000000,-0.756272,195679.012346
5,0.074074,-0.125000,0.009601,0.000000,1.623061,226436.781609
6,-0.127660,-0.444444,-0.005564,0.000000,0.041888,178048.780488
7,0.015873,0.000000,0.000000,0.000000,0.000000,212500.000000
8,0.175676,-0.176471,-0.003988,0.000000,-0.295754,226436.781609
9,0.100000,-0.250000,0.002035,0.000000,0.318530,213409.090909
